In [1]:
# !wget https://cdn.commonvoice.mozilla.org/cv-corpus-5.1-2020-06-22/id.tar.gz
# !tar -zxf id.tar.gz

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part2.tar
# !mkdir part1-v2
# !tar -xf semisupervised-26-02-2021-part2.tar -C part1-v2

In [3]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part3.tar
# !mkdir part2-v2
# !tar -xf semisupervised-26-02-2021-part3.tar -C part2-v2

In [4]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part4.tar
# !mkdir part3-v2
# !tar -xf semisupervised-26-02-2021-part4.tar -C part3-v2

In [5]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part1.tar
# !tar -xf semisupervised-24-03-2021-part1.tar

In [6]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part2.tar
# !tar -xf semisupervised-24-03-2021-part2.tar

In [7]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part3.tar
# !tar -xf semisupervised-24-03-2021-part3.tar

In [8]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech-bahasa.zip
# !unzip speech-bahasa.zip

In [9]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/streaming.zip -O wikipedia-asr.zip
# !unzip wikipedia-asr.zip

In [10]:
# wget https://f000.backblazeb2.com/file/malaya-speech-model/data/news-speech.zip
# wget https://f000.backblazeb2.com/file/malaya-speech-model/collections/transcript-news.json
# unzip news-speech.zip -d news

In [11]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/trainset-audiobook.tar.gz
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/text-audiobook.tar.gz
# !tar -xf trainset-audiobook.tar.gz
# !tar -zxf text-audiobook.tar.gz

In [12]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [13]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import json

In [14]:
base_directory = '/home/husein/speech-bahasa'

In [15]:
df = pd.read_csv(f'{base_directory}/cv-corpus-5.1-2020-06-22/id/validated.tsv', sep = '\t')
df = df[(df['sentence'].str.len() > 5) & (df['sentence'].str.count(' ') > 0)]
print(df.shape)

(7490, 10)


In [16]:
id_commonvoice = []
for i in range(len(df)):
    p = f"{base_directory}/cv-corpus-5.1-2020-06-22/id/clips/{df['path'].iloc[i]}"
    t = df['sentence'].iloc[i]
    if len(t) < 5:
        continue
    id_commonvoice.append((p, t))

len(id_commonvoice)

7490

In [17]:
malay = glob(f'{base_directory}/part*/output-wav/*.wav')
len(malay)

1008453

In [18]:
malay.extend(glob(f'{base_directory}/part*/semisupervised/output-wav/*.wav'))
len(malay)

1322836

In [19]:
khalil = glob(f'{base_directory}/tolong-sebut/*.wav')
mas = glob(f'{base_directory}/sebut-perkataan-woman/*.wav')
husein = glob(f'{base_directory}/sebut-perkataan-man/*.wav')
len(khalil), len(mas), len(husein)

(565, 200, 698)

In [20]:
khalils = []
for i in tqdm(khalil[:-int(len(khalil) * 0.05)]):
    try:
        t = i.split('/')[-1].replace('.wav','')
        text = f'tolong sebut {t}'
        khalils.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 537/537 [00:00<00:00, 5034.84it/s]


In [21]:
mass = []
for i in tqdm(mas[:-int(len(mas) * 0.05)]):
    try:
        t = i.split('/')[-1].replace('.wav','')
        text = f'sebut perkataan {t}'
        mass.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 190/190 [00:00<00:00, 193426.64it/s]


In [22]:
huseins = []
for i in tqdm(husein[:-int(len(husein) * 0.05)]):
    try:
        t = i.split('/')[-1].replace('.wav','')
        text = f'sebut perkataan {t}'
        huseins.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 664/664 [00:00<00:00, 223480.81it/s]


In [23]:
malays = []
for i in tqdm(malay):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        if len(text) < 3:
            continue
        malays.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 1322836/1322836 [01:04<00:00, 20405.69it/s]


In [24]:
wikipedia = []
wavs = glob(f'{base_directory}/streaming/*wav')
for i in tqdm(wavs[:-int(len(wavs) * 0.05)]):
    text = os.path.split(i)[1].replace('.wav', '')
    wikipedia.append((i, text))
    
len(wikipedia)

100%|██████████| 2743/2743 [00:00<00:00, 13014.35it/s]


2743

In [25]:
news = []
wavs = glob(f'{base_directory}/news/audio/*wav')

with open(f'{base_directory}/transcript-news.json') as fopen:
    transcript_news = json.load(fopen)
    
for i in tqdm(wavs[:-int(len(wavs) * 0.05)]):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript_news[int(index)]
    news.append((i, text))

100%|██████████| 2044/2044 [00:00<00:00, 311751.18it/s]


In [26]:
audiobook = []
wavs = glob(f'{base_directory}/combined/*wav')
for i in tqdm(wavs):
    t = '/'.join(i.split('<>')[1:])
    t = t.split('.wav')[0]
    t = t.replace('output-wav', 'output-text')
    with open(f'{base_directory}/text-audiobook/{t}.wav.txt') as fopen:
        text = fopen.read()
    audiobook.append((i, text))

100%|██████████| 64341/64341 [00:03<00:00, 19354.11it/s]


In [27]:
df = pd.read_csv(f'{base_directory}/haqkiem/metadata.csv', header = None, sep = '|')
txts = df.values.tolist()
haqkiem = []
for f in tqdm(txts[:-int(len(txts) * 0.05)]):
    text = f[1]
    text = text.split('.,,')[0]
    f = f[0]
    r = f'{base_directory}/haqkiem/{f}.wav'
    haqkiem.append((r, text))

100%|██████████| 4080/4080 [00:00<00:00, 202914.10it/s]


In [28]:
# import IPython.display as ipd

# ipd.Audio(audiobook[0][0])

In [29]:
audios = id_commonvoice + malays + wikipedia + news + audiobook + haqkiem + khalils + mass + huseins
audios, texts = zip(*audios)

In [30]:
len(texts)

1404435

In [31]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [32]:
processed_text = [preprocessing_text(t) for t in tqdm(texts)]

100%|██████████| 1404435/1404435 [01:58<00:00, 11883.65it/s]


In [33]:
audios[-100:], processed_text[-100:]

(('/home/husein/speech-bahasa/sebut-perkataan-man/alku.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/abdiad.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/amendmen.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/agakkan.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/alatan.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/acaranya.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/mengakuri.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/amorfus.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/keaktifan.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/abah.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/ambar.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/mengamankan.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/afasia.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/amberol.wav',
  '/home/husein/speech-bahasa/sebut-perkataan-man/alkasyaf.wav',
  '/home/husein/speech-bahasa/sebut-perk

In [34]:
with open('bahasa-asr-train.json', 'w') as fopen:
    json.dump({'X': audios, 'Y':processed_text}, fopen)

In [35]:
# import malaya_speech

In [36]:
# tokenizer = malaya_speech.subword.generate_tokenizer(processed_text, max_subword_length = 3)

In [37]:
# malaya_speech.subword.save(tokenizer, 'transducer.subword')

In [38]:
# tokenizer = malaya_speech.subword.load('transducer.subword')
# malaya_speech.subword.encode(tokenizer, 'i hate', add_blank = True)

In [39]:
# malaya_speech.subword.decode(tokenizer, [0, 2, 133, 875])

In [40]:
# from pydub import AudioSegment
# import numpy as np

# sr = 16000

# def mp3_to_wav(file, sr = sr):
#     audio = AudioSegment.from_file(file)
#     audio = audio.set_frame_rate(sr).set_channels(1)
#     sample = np.array(audio.get_array_of_samples())
#     return malaya_speech.astype.int_to_float(sample), sr

# def generator(maxlen = 18, min_length_text = 2):
#     for i in tqdm(range(len(audios))):
#         try:
#             if audios[i].endswith('.mp3'):
#                 wav_data, _ = mp3_to_wav(audios[i])
#             else:
#                 wav_data, _ = malaya_speech.load(audios[i])
                
#             if (len(wav_data) / sr) > maxlen:
#                 # print(f'skipped audio too long {audios[i]}')
#                 continue
                
#             if len(processed_text[i]) < min_length_text:
#                 print(f'skipped text too short {audios[i]}')
#                 continue    

#             yield {
#                 'waveforms': wav_data.tolist(),
#                 'waveform_lens': [len(wav_data)],
#                 'targets': malaya_speech.subword.encode(tokenizer, processed_text[i], add_blank = False),
#             }
#         except Exception as e:
#             print(e)
            
# generator = generator()

In [41]:
# import os
# import tensorflow as tf

# os.system('rm bahasa-asr/data/*')
# DATA_DIR = os.path.expanduser('bahasa-asr/data')
# tf.gfile.MakeDirs(DATA_DIR)

In [42]:
# shards = [{'split': 'train', 'shards': 1000}]

In [43]:
# import malaya_speech.train as train

# train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')